In [1]:
import numpy as np
import pandas as pd
import os
import pickle

from surprise import SVD, Dataset, dataset, Reader
from collections import defaultdict

In [2]:
df = pd.read_csv('data/CNPC_1401-1509_DI_v1_1_2016-03-01.csv')
df = df.drop('registered', 1)
df = df.drop('viewed', 1)
df = df.drop('explored', 1)
df = df.drop('grade', 1)
df = df.drop('grade_reqs', 1)
df = df.drop('course_reqs', 1)
df = df.drop('final_cc_cname_DI', 1)
df = df.drop('primary_reason', 1)
df = df.drop('learner_type', 1)
df = df.drop('expected_hours_week', 1)
df = df.drop('LoE_DI', 1)
df = df.drop('age_DI', 1)
df = df.drop('gender', 1)
df = df.drop('start_time_DI', 1)
df = df.drop('course_start', 1)
df = df.drop('course_end', 1)
df = df.drop('last_event_DI', 1)
df = df.drop('nevents', 1)
df = df.drop('ndays_act', 1)
df = df.drop('ncontent', 1)
df = df.drop('nforum_posts', 1)
df = df.drop('course_length', 1)
df = df.drop('discipline', 1)
# df = df[pd.notnull(df['completed_%'])]
df['completed_%'] = df['completed_%'].fillna(0)
df['completed_%'] = df['completed_%'] * 100
# df.index = pd.to_datetime(df.course_end)
# df['year'] = df.index.year
# df['quarter'] = df.index.quarter
# df['course_end'] = pd.to_datetime(df['course_end']).dt.to_period('Q')

# df['year'] = df['course_end'].dt.year
# df['quarter'] = df['course_end'].dt.quarter

# df['timestamp'] = df['course_end']
# df['timestamp'] = pd.to_datetime(df['course_end'].str.replace(' ', '')) + pd.offsets.QuarterEnd(0)
# print(df.head())

C:\Users\jason\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def get_top_n(predictions, n=10):

    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [4]:
class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['userid_DI'], df['course_id_DI'], df['completed_%'])]
        self.reader=reader
        
reader = Reader(line_format='user item rating', rating_scale=(0, 100))

data = MyDataset(df, reader)

algo = SVD()

# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

algo.fit(trainset)
# test = [(832960083, 832380973, None)]
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
# for uid, user_ratings in top_n.items():
#     print(uid, [iid for (iid, _) in user_ratings])
# print(top_n)
# print(top_n[832318743])

In [22]:
binary_file = open('top_10.bin',mode='wb')
pickled_top_ten = pickle.dump(top_n, binary_file, protocol=2)
binary_file.close()